In [94]:
import open3d as o3d
import numpy as np
import laspy
import matplotlib.pyplot as plt
import imageio
import os
import time

In [95]:
import time

In [96]:
# Load the LAS file
las_file = r"C:\Users\gedsloov\OneDrive - UGent\UGent\PhD\05_Research\02_ICOS\02_Data\ICOS_BRA_for_presentation\ICOS_BRA_CP 0.100 m.las"
las = laspy.read(las_file)

In [97]:
las

<LasData(1.4, point fmt: <PointFormat(3, 6 bytes of extra dims)>, 6510282 points, 2 vlrs)>

In [98]:
# Extract coordinates
points = np.vstack((las.x, las.y, las.z)).T  # X, Y, Z

In [99]:
# Extract RGB colors (Normalize to 0-1 for Open3D)
colors = np.vstack((las.red, las.green, las.blue)).T / 65535  # LAS stores RGB in 16-bit


In [100]:
# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

In [101]:
# Downsampling (if necessary)
if len(points) > 1_000_000:  # Reduce if the file is too large
    pcd = pcd.voxel_down_sample(voxel_size=0.05)  # Adjust voxel size as needed

In [102]:
# Set up rotation angles
num_frames = 60  # Number of frames for full rotation
rotation_axis = [0, 0, 1]  # Rotate around Z-axis
output_folder = "rotation_frames"
os.makedirs(output_folder, exist_ok=True)

In [103]:
# Create visualizer
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False)
vis.add_geometry(pcd)

True

In [104]:
# Get the view control
view_control = vis.get_view_control()
camera_params = view_control.convert_to_pinhole_camera_parameters()


In [105]:
# 🔥 Reset the camera first to a valid default
view_control.set_front([1, -1, 0])  # Side view but slightly rotated
view_control.set_up([0, 0, 1])  # Keep Z as the up direction
view_control.set_lookat(pcd.get_center())  # Center the camera on the point cloud
view_control.set_zoom(0.8)  # Zoom out a bit to fit the whole point cloud


In [106]:
# Force Open3D to apply these camera settings
vis.poll_events()
vis.update_renderer()
time.sleep(1)  # ✅ Give Open3D time to render

In [107]:
# Generate frames
for i in range(num_frames):
    angle = (i / num_frames) * 360  # Rotate full 360°
    R = pcd.get_rotation_matrix_from_axis_angle(np.radians(angle) * np.array(rotation_axis))
    pcd_rotated = pcd.rotate(R, center=pcd.get_center())

    vis.update_geometry(pcd_rotated)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.5)  # ✅ Ensures rendering completes before capture

    # Save frame
    image_path = f"{output_folder}/frame_{i:03d}.png"
    vis.capture_screen_image(image_path)
    print(f"Saved frame {i+1}/{num_frames}")

vis.destroy_window()


Saved frame 1/60
Saved frame 2/60
Saved frame 3/60
Saved frame 4/60
Saved frame 5/60
Saved frame 6/60
Saved frame 7/60
Saved frame 8/60
Saved frame 9/60
Saved frame 10/60
Saved frame 11/60
Saved frame 12/60
Saved frame 13/60
Saved frame 14/60
Saved frame 15/60
Saved frame 16/60
Saved frame 17/60
Saved frame 18/60
Saved frame 19/60
Saved frame 20/60
Saved frame 21/60
Saved frame 22/60
Saved frame 23/60
Saved frame 24/60
Saved frame 25/60
Saved frame 26/60
Saved frame 27/60
Saved frame 28/60
Saved frame 29/60
Saved frame 30/60
Saved frame 31/60
Saved frame 32/60
Saved frame 33/60
Saved frame 34/60
Saved frame 35/60
Saved frame 36/60
Saved frame 37/60
Saved frame 38/60
Saved frame 39/60
Saved frame 40/60
Saved frame 41/60
Saved frame 42/60
Saved frame 43/60
Saved frame 44/60
Saved frame 45/60
Saved frame 46/60
Saved frame 47/60
Saved frame 48/60
Saved frame 49/60
Saved frame 50/60
Saved frame 51/60
Saved frame 52/60
Saved frame 53/60
Saved frame 54/60
Saved frame 55/60
Saved frame 56/60
S

In [108]:
# Create GIF with slower animation
frame_files = sorted([f"{output_folder}/{f}" for f in os.listdir(output_folder) if f.endswith(".png")])
frames = [imageio.imread(f) for f in frame_files]
gif_path = "rotating_pointcloud_slightly_above.gif"
imageio.mimsave(gif_path, frames, fps=5)  # Slow down by reducing FPS

print(f"GIF saved as {gif_path}")

C:\Users\gedsloov\AppData\Local\Temp\ipykernel_4760\2581745818.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames = [imageio.imread(f) for f in frame_files]


GIF saved as rotating_pointcloud_slightly_above.gif


In [109]:
import open3d as o3d
import numpy as np
import laspy
import imageio.v2 as imageio
import os
import time

# Load the LAS file
las_file = r"C:\Users\gedsloov\OneDrive - UGent\UGent\PhD\05_Research\02_ICOS\02_Data\ICOS_BRA_for_presentation\ICOS_BRA_CP 0.100 m.las"
las = laspy.read(las_file)

# Extract coordinates
points = np.vstack((las.x, las.y, las.z)).T  # X, Y, Z

# Extract RGB colors (Normalize to 0-1 for Open3D)
colors = np.vstack((las.red, las.green, las.blue)).T / 65535

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Downsampling if needed
if len(points) > 10_000_000:  # Keep ~10M points max
    pcd = pcd.voxel_down_sample(voxel_size=0.05)

# Set up rotation and camera
num_frames = 60  # More frames = smoother rotation
rotation_axis = [0, 0, 1]  # Rotate around Z-axis
output_folder = "rotation_frames"
os.makedirs(output_folder, exist_ok=True)

# Create Open3D visualizer
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False)
vis.add_geometry(pcd)

# 🔥 Set PURE Side View (No Above Angle Yet)
view_control = vis.get_view_control()
view_control.set_front([1, 0, 0])  # Side view along X-axis
view_control.set_up([0, 0, 1])  # Z-axis is up
view_control.set_lookat(pcd.get_center())  # Center on point cloud
view_control.set_zoom(0.8)  # Slight zoom out to fit the whole cloud

# Force Open3D to apply these camera settings
vis.poll_events()
vis.update_renderer()
time.sleep(1)  # Allow Open3D time to render

# Generate frames
for i in range(num_frames):
    angle = (i / num_frames) * 360  # Rotate full 360°
    R = pcd.get_rotation_matrix_from_axis_angle(np.radians(angle) * np.array(rotation_axis))
    pcd_rotated = pcd.rotate(R, center=pcd.get_center())

    vis.update_geometry(pcd_rotated)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.5)  # Ensures rendering completes before capture

    # Save frame
    image_path = f"{output_folder}/frame_{i:03d}.png"
    vis.capture_screen_image(image_path)
    print(f"Saved frame {i+1}/{num_frames}")

vis.destroy_window()

# Create GIF with slower animation
frame_files = sorted([f"{output_folder}/{f}" for f in os.listdir(output_folder) if f.endswith(".png")])
frames = [imageio.imread(f) for f in frame_files]
gif_path = "rotating_pointcloud_sideview.gif"
imageio.mimsave(gif_path, frames, fps=5)  # Slow down by reducing FPS

print(f"GIF saved as {gif_path}")


Saved frame 1/60
Saved frame 2/60
Saved frame 3/60
Saved frame 4/60
Saved frame 5/60
Saved frame 6/60
Saved frame 7/60
Saved frame 8/60
Saved frame 9/60
Saved frame 10/60
Saved frame 11/60
Saved frame 12/60
Saved frame 13/60
Saved frame 14/60
Saved frame 15/60
Saved frame 16/60
Saved frame 17/60
Saved frame 18/60
Saved frame 19/60
Saved frame 20/60
Saved frame 21/60
Saved frame 22/60
Saved frame 23/60
Saved frame 24/60
Saved frame 25/60
Saved frame 26/60
Saved frame 27/60
Saved frame 28/60
Saved frame 29/60
Saved frame 30/60
Saved frame 31/60
Saved frame 32/60
Saved frame 33/60
Saved frame 34/60
Saved frame 35/60
Saved frame 36/60
Saved frame 37/60
Saved frame 38/60
Saved frame 39/60
Saved frame 40/60
Saved frame 41/60
Saved frame 42/60
Saved frame 43/60
Saved frame 44/60
Saved frame 45/60
Saved frame 46/60
Saved frame 47/60
Saved frame 48/60
Saved frame 49/60
Saved frame 50/60
Saved frame 51/60
Saved frame 52/60
Saved frame 53/60
Saved frame 54/60
Saved frame 55/60
Saved frame 56/60
S